In [1]:
import numpy as np
import pandas as pd
import sklearn.metrics as skm
from sklearn.preprocessing import MinMaxScaler
from time import time

In [2]:
# t1 = pd.read_csv('pearson.csv')

In [3]:
# scaler = MinMaxScaler()
# scaler.fit(t1)

In [4]:
# print(scaler.transform(t1))

In [5]:
test = pd.read_csv('test.csv',dtype=int)
train = pd.read_csv('train.csv')
test

,iId,uId,rating
0,1460,303,3
1,660,633,3
2,274,502,3
3,553,35,3
4,1351,377,4
...,...,...,...
317221,1323,861,4
317222,1126,98,4
317223,1602,561,3
317224,1122,890,4


In [6]:
# test['rating'] = test['rating'].round().astype(int)
train['uId'] = train['uId'].round().astype(int)
train['iId'] = train['iId'].round().astype(int)
train['rating'] = train['rating'].round().astype(int)
train = train.to_numpy()
test = test.to_numpy()


In [7]:
matrix = np.zeros((943,1683),dtype=int)
for i in train:
        matrix[i[1]-1,i[0]-1] = i[2]

In [8]:
def PredRating(user,item,similarity):
    try:
        top = similarity[user].argsort()[1:100]
    except IndexError:
        sum,count = 0,0
        for j in range(1682):
            if matrix[user,j] != 0:
                count+=1
                sum+=1
        return sum/count
    temp,avgUh,simi = [],[],[]
    for i in top:
        if matrix[i,item] !=0:
            temp.append(i)
            simi.append(similarity[user,i])
    temp.append(user)
    for i in temp:
        sum,count = 0,0
        for j in range(1682):
            if matrix[i,j] != 0:
                count+=1
                sum+=matrix[i,j]
        avgUh.append(sum/count)
    avgU = np.nan_to_num(np.array(avgUh.pop()))
    temp.pop()
    if len(temp)==0:
        return avgU
    simi = np.nan_to_num(np.array(simi))
    num = (simi*(avgUh-avgU)).sum()
    den = simi.sum()
    # for i in range(len(temp)):
    #     num+=similarity[user,temp[i]]*avgUh[i]
    #     den+=similarity[user,temp[i]]
    try:
        result = int(round((avgU+num/den),0))
    except ZeroDivisionError:
        result = 0
    return result



In [9]:
pip = pd.read_csv('mpip.csv').to_numpy()
pipPred = np.zeros(len(test),dtype=int)

In [10]:
# %%time
j = 0
for i in test:
    pipPred[j] = PredRating(i[1]-1,i[0]-1,pip)
    j+=1
    # if j==100:
    #     break


In [18]:
test = pd.read_csv('test.csv')
pipPred = pd.DataFrame(pipPred)
pipPred.columns = ['mpip']
frames = [test,pipPred]
t = pd.concat(frames,axis=1)
t.to_csv('mpipRatings1.csv',sep=',',index=False)

In [16]:
pipPred.columns = ['mpip']

In [17]:
pipPred

,mpip
0,4
1,4
2,4
3,4
4,4
...,...
317221,4
317222,4
317223,4
317224,4
